In [1]:
%pip install azure-storage-file-datalake

In [13]:
import io, os, uuid
from azure.identity import DefaultAzureCredential, ManagedIdentityCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import pandas as pd

In [2]:
ACCOUNT_NAME = "xxx"
ACCOUNT_KEY = ""
CONTAINER_NAME = "tlfs"
FILE_PATH = 'poc/parquet/nyc_yellow_taxi_trips/puYear=2001/puMonth=1/'
FILE = 'part-00493-127a558e-137b-4f6b-8c6e-904dc74a264e.c000.snappy.parquet'

In [10]:
from azure.storage.filedatalake import DataLakeServiceClient

def initialize_storage_account(storage_account_name, storage_account_key):
    try:
        global service_client

        #credential = AccountSasCredentials(storage_account_name, storage_account_key)
        #default_credential = DefaultAzureCredential()

        service_client = DataLakeServiceClient(account_url=f"https://{storage_account_name}.dfs.core.windows.net")
    
    except Exception as e:
        print(f"Error initializing Data Lake service client: {e}")
        raise

import pyarrow as pa
import io

def read_parquet_from_adls(container_name, parquet_file_path):
    # Get a reference to the container
    container_client = service_client.get_container_client(container_name)

    # Get a reference to the file
    file_client = container_client.get_file_client(parquet_file_path)

    # Download the data as bytes
    byte_stream = io.BytesIO()
    downloader = file_client.download()
    downloader.readinto(byte_stream)

    # Read the Parquet file
    byte_stream.seek(0)  # Reset the stream position
    parquet_file = pa.ipc.open_file(byte_stream)
    table = parquet_file.read_all()

    return table

In [22]:
temp_path = FILE_PATH + FILE
print(temp_path)
parquet_path= f'abfss://{CONTAINER_NAME}@{ACCOUNT_NAME}.dfs.core.windows.net/{temp_path}'
print(parquet_path)
storage_opts={'account_key':'TODO'}
#parquet_path = "poc/parquet/nyc_yellow_taxi_trips/puYear=2001/puMonth=1/part-00493-127a558e-137b-4f6b-8c6e-904dc74a264e.c000.snappy.parquet"
#df = pd.read_parquet(f'abfss://{CONTAINER_NAME}@{ACCOUNT_NAME}.dfs.core.windows.net/{parquet_path}')
df = pd.read_parquet(path=parquet_path, storage_options=storage_opts)
display(df)

In [11]:
initialize_storage_account(ACCOUNT_NAME, ACCOUNT_KEY)

table = read_parquet_from_adls(CONTAINER_NAME, FILE_PATH + FILE)
print(table.to_pandas())

In [23]:
account_url = f"https://{ACCOUNT_NAME}.blob.core.windows.net"

try:
    print("Azure Blob Storage Python quickstart sample")
    mi_credential = ManagedIdentityCredential(client_id="xxx")
    default_credential = DefaultAzureCredential()
    # Create the BlobServiceClient object
    #blob_service_client = BlobServiceClient(account_url, credential=cred)
    #stream = blob_service_client.download_blob()# Instantiate a ContainerClient
    container_client = blob_service_client.get_container_client(CONTAINER_NAME)

    

except ResourceNotFoundError:
    print("No blob found.")
except Exception as ex:
    print('Exception:')
    print(ex)

In [27]:
blob_service_client = BlobServiceClient.from_connection_string(conn_str=conn_str)

In [29]:
def download_blob_to_stream(blob_service_client: BlobServiceClient, container_name):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=PARQUET_FILE_PATH)

    # readinto() downloads the blob contents to a stream and returns the number of bytes read
    stream = io.BytesIO()
    num_bytes = blob_client.download_blob().readinto(stream)
    print(f"Number of bytes: {num_bytes}")

In [31]:
download_blob_to_stream(blob_service_client,CONTAINER_NAME)